In [1]:
%gui qt
%matplotlib qt

import numpy as np
from pprint import pprint
from importlib import reload

In [7]:
from plottr.data.datadict import DataDict

from plottr import node; reload(node)
from plottr.node import Node, NodeWidget, DataSelector

# Tool functions

In [3]:
def testdata_1d(nvals):
    x = np.linspace(0, 10, nvals)
    y = np.cos(x)
    z = np.cos(x)**2
    d = DataDict(
        x = {'values' : x},
        y = {'values' : y, 'axes' : ['x']},
        z = {'values' : z, 'axes' : ['x']},
    )
    return d

def testdata_3d(nx, ny, nz):
    x = np.linspace(0, 10, nx)
    y = np.linspace(-5, 5, ny)
    z = np.arange(nz)
    xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
    dd = np.cos(xx) * np.sin(yy) + np.random.rand(*zz.shape)
    dd2 = np.sin(xx) * np.cos(yy) + np.random.rand(*zz.shape)
    d = DataDict(
        x = dict(values=xx.reshape(-1)),
        y = dict(values=yy.reshape(-1)),
        z = dict(values=zz.reshape(-1)),
        data = dict(values=dd.reshape(-1), axes=['x', 'y', 'z']),
        more_data = dict(values=dd2.reshape(-1), axes=['x', 'y', 'z']),
        different_data = dict(values=dd2.T.reshape(-1), axes=['z', 'y', 'x'])
    )
    return d

# Basic data selection

In [ ]:
data1 = testdata_3d(3, 5, 5)
datasrc = Node()
datasrc.verbose = True
datasrc.data = data1

In [ ]:
datasel = DataSelector()
datasel.verbose = True
datasel.setInput(datasrc)

In [ ]:
datasel.dataName = 'more_data'
datasel.slices = dict(y=np.s_[::2], x=np.s_[:2:])
datasel.axesOrder = dict(x=1,z=2,y=0)

# Basic checking

In [ ]:
data1 = testdata_3d(3, 5, 5)

datasrc = Node()
datasrc.verbose = False

datasel = DataSelector()
datasel.verbose = False
datasel.setInput(datasrc)

In [ ]:
datasrc.data = data1
datasel.dataName = ['data', 'more_data']

In [ ]:
datasel.data

# Data Selector Widget

In [19]:
from PyQt5.QtWidgets import QWidget, QVBoxLayout, QFormLayout
from PyQt5.QtCore import QObject, QThread, pyqtSignal, pyqtSlot


class NodeControlWidget(QWidget):
    
    def __init__(self, parent=None, **kw):
        super().__init__(parent=parent, **kw)


class DataSelectorWidget(NodeControlWidget):
    
    newDataStructure = pyqtSignal()
    
    def __init__(self, parent=None, node=None, **kw):
        super().__init__(parent=parent, **kw)
        
        # this is more general!
        if node is None:
            self.node = DataSelector()
        else:
            self.node = node
        self.node.dataProcessed.connect(self.dataUpdate)
        
        self._axes = []
                
        axLayout = QFormLayout(self)
        
        
        
    def dataUpdate(self):
        # TODO: is this too agressive?
        newAxes = self.node.axesList(self.node.data, self.node.dataName)
        if newAxes != self._axes:
            self._axes = newAxes
            self.newDataStructure.emit()

In [20]:
data1 = testdata_3d(3, 5, 5)

datasrc = Node()
datasrc.data = data1

w = DataSelectorWidget()
w.show()

w.node.setInput(datasrc)

In [16]:
w.node.dataName = ['data', 'more_data']

['x', 'y', 'z']


# Updating a 1D plot

In [ ]:
data1 = testdata_1d(21)
datasrc = Node()
datasrc.data = data1

lineplotdata = LinePlotData()
lineplotdata.setInput(datasrc)

lp = LinePlot(None)
lp.setInput(lineplotdata)
lp.show()

In [ ]:
lineplotdata.xaxisName = 'x'
lineplotdata.traceNames = ['z']